In [ ]:
import os
import sys
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai.chat_models import AzureChatOpenAI        
from langchain_openai.embeddings import AzureOpenAIEmbeddings   
import json

In [2]:
## Paths in the project

# Base Folder:
base_folder = os.path.dirname(os.getcwd())

# Answers path
answers_path = os.path.join(base_folder, 'Answers')

# Vector_DBs path
vector_dbs_path = os.path.join(base_folder, 'Vector_DBs')

# Code path
code_path = os.path.join(base_folder, 'Code')

# AI_Prepositions path
ai_prepositions_path = os.path.join(base_folder, 'AI_Prepositions')

# Corpus path
corpus_path = os.path.join(base_folder, 'Corpus')

# Ground_Truth path
ground_truth_path = os.path.join(base_folder, 'Ground_Truth')

# RAGAS_Results path
ragas_results_path = os.path.join(base_folder, 'RAGAS_Results')

In [ ]:
# Configurar la API de Azure

os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_KEY" # Your Key

azure_configs = {
    "base_url": "YOUR_URL", # Your endpoint
    "model_deployment": "YOUR_MODEL", # Your model_deployment (LLM)
    "model_name": "YOUR_MODEL_NAME", # Your model_name (LLM)
    "embedding_deployment": "YOUR_MODEL", # Your model_deployment (embedding)
    "embedding_name": "YOUR_MODEL_NAME", # Your model_name (embedding)
}

In [3]:
# List of metrics
metrics = [answer_relevancy, context_precision, context_recall, faithfulness]

In [ ]:
# Configure the LLM and Embeddings using wrappers
azure_llm = AzureChatOpenAI(
    openai_api_version="YOUR API_version", # Your API_version
    azure_endpoint=azure_configs["base_url"], 
    azure_deployment=azure_configs["model_deployment"], 
    model=azure_configs["model_name"], 
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_version="YOUR API_version", # Your API_version
    azure_endpoint=azure_configs["base_url"],
    azure_deployment=azure_configs["embedding_deployment"],
    model=azure_configs["embedding_name"],
)


#  Wrap the Azure LLM instance
llm = LangchainLLMWrapper(azure_llm)
# Wrap the Azure embeddings model
embeddings = LangchainEmbeddingsWrapper(azure_embeddings)

In [4]:
# We need the .json file
# Import the needed .json file
with open(os.path.join(ground_truth_path, "mistral_ai_truth.json"), "r", encoding="utf-8") as f: # Change the name of the file
    data = json.load(f)

In [6]:
# Transform it to HuggingFace Dataset format
dataset = Dataset.from_list(data)

In [ ]:
# Evaluate with the desired metrics
results = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
    llm=llm,
    embeddings=embeddings
)

# Display results
print("===== Métricas RAGAS con GPT-4o =====") # RAGAS metrics with GPT-4o
print(results)

Evaluating: 100%|██████████| 12/12 [00:22<00:00,  1.90s/it]


===== Métricas RAGAS con GPT-4o =====
{'faithfulness': 0.9872, 'answer_relevancy': 0.5830, 'context_precision': 0.2611, 'context_recall': 0.3026}


In [ ]:
# Save the result as df
df = results.to_pandas()

# Export the result
df.to_excel(os.path.join(ragas_results_path, "mistral_ai_RAGAS_score.xlsx"), index=False) # Change the name according with the model that you are using